In [1]:
from pyspark_llap import HiveWarehouseSession
hive = HiveWarehouseSession.session(spark).build()
hive.showDatabases().show(10)

+------------------+
|     database_name|
+------------------+
|           default|
|information_schema|
|               pdi|
|            poc_db|
|               sys|
|           twitter|
+------------------+



In [2]:
hive.setDatabase("default")
hive.createTable("web_sales").ifNotExists().\
    column("sold_time_sk", "bigint").column("ws_ship_date_sk", "bigint").create()
hive.describeTable("web_sales").show()
hive.dropTable("web_sales", True, True)

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|   sold_time_sk|   bigint|       |
|ws_ship_date_sk|   bigint|       |
+---------------+---------+-------+



In [3]:
df = spark.read.csv("/data/examples/Employees.tsv", 
                    header=False, sep='\t', mode="DROPMALFORMED", inferSchema=True
                   ).toDF("name", "title", "department", "salary")
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: integer (nullable = true)



In [4]:
df.write.format(HiveWarehouseSession().HIVE_WAREHOUSE_CONNECTOR)\
    .mode("overwrite").option("table", "employees").save()

In [5]:
df = hive.executeQuery("select * from employees")
df.toPandas().head(10)

,name,title,department,salary
0,"AARON, ELVIA J",WATER RATE TAKER,WATER MGMNT,81000
1,"AARON, JEFFERY M",POLICE OFFICER,POLICE,74628
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,FLEET MANAGEMNT,77280
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,96276
4,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,WATER MGMNT,84032
5,"ABBATE, TERRY M",POLICE OFFICER,POLICE,79926
6,"ABBATEMARCO, JAMES J",FIREFIGHTER,FIRE,77238
7,"ABBOTT, BETTY L",FOSTER GRANDPARENT,FAMILY & SUPPORT,2756
8,"ABBOTT, LYNISE M",CLERK III,FAMILY & SUPPORT,38568
9,"ABBOTT, SAM J",ELECTRICAL MECHANIC,TRANSPORTN,84032
